In [ ]:
#Import all data packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

from nltk.corpus import stopwords
from nltk import word_tokenize 
from nltk.stem import WordNetLemmatizer
import re 
import string
from tqdm import tqdm
from emotion import*
import emoji

import statsmodels.api as sm
import scipy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.metrics import roc_auc_score, auc, roc_curve, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from scipy.stats import sem
from sklearn.metrics import precision_recall_curve

import shap

## Data Collection
Using Twitter's API to search for the FakeNewsNet tweet IDs resulted in four CSV files. 
- Filter out non-english language tweets and any tweets that are replies, retweets or quotes
- Combine four files into one dataframe 

In [ ]:
#Import relevant data
col_list = [0,2,3,4,10,11,16, 24, 25, 26, 50, 51, 53]
politifact_fake_df = pd.read_csv('politifact_fake_flat.csv', 
                                 usecols=col_list, index_col=0)
politifact_real_df = pd.read_csv('politifact_real_flat.csv', 
                                 usecols=col_list, index_col=0)
gossipcop_fake_df = pd.read_csv('gossipcop_fake_flat.csv', 
                                usecols=col_list, index_col=0)
gossipcop_real_df = pd.read_csv('gossipcop_real_flat.csv', 
                                usecols=col_list, index_col=0)

In [ ]:
#Rename columns
df_names = politifact_fake_df.rename({'referenced_tweets.replied_to.id': 'reply', 
                                      'referenced_tweets.retweeted.id': 'retweet', 
                                      'referenced_tweets.quoted.id': 'quote', 
                                      'entities.hashtags': 'hashtags', 
                                      'author.public_metrics.followers_count': 'followers', 
                                      'author.public_metrics.following_count': 'following', 
                                      'author.public_metrics.tweet_count': 'tweet_count', 
                                      'public_metrics.retweet_count': 'num_retweets', 
                                      'entities.mentions': 'mentions', 
                                      'entities.urls':'urls'}, axis=1)
pr_names = politifact_real_df.rename({'referenced_tweets.replied_to.id': 'reply', 
                                      'referenced_tweets.retweeted.id': 'retweet', 
                                      'referenced_tweets.quoted.id': 'quote', 
                                      'entities.hashtags': 'hashtags', 
                                      'author.public_metrics.followers_count': 'followers', 
                                      'author.public_metrics.following_count': 'following', 
                                      'author.public_metrics.tweet_count': 'tweet_count', 
                                      'public_metrics.retweet_count': 'num_retweets', 
                                      'entities.mentions': 'mentions', 
                                      'entities.urls':'urls'}, axis=1)
gf_names = gossipcop_fake_df.rename({'referenced_tweets.replied_to.id': 'reply', 
                                      'referenced_tweets.retweeted.id': 'retweet', 
                                      'referenced_tweets.quoted.id': 'quote', 
                                      'entities.hashtags': 'hashtags', 
                                      'author.public_metrics.followers_count': 'followers', 
                                      'author.public_metrics.following_count': 'following', 
                                      'author.public_metrics.tweet_count': 'tweet_count', 
                                      'public_metrics.retweet_count': 'num_retweets', 
                                      'entities.mentions': 'mentions', 
                                      'entities.urls':'urls'}, axis=1)
gr_names = gossipcop_real_df.rename({'referenced_tweets.replied_to.id': 'reply', 
                                      'referenced_tweets.retweeted.id': 'retweet', 
                                      'referenced_tweets.quoted.id': 'quote', 
                                      'entities.hashtags': 'hashtags', 
                                      'author.public_metrics.followers_count': 'followers', 
                                      'author.public_metrics.following_count': 'following', 
                                      'author.public_metrics.tweet_count': 'tweet_count', 
                                      'public_metrics.retweet_count': 'num_retweets', 
                                      'entities.mentions': 'mentions', 
                                      'entities.urls':'urls'}, axis=1)

In [ ]:
#Function to filter out non-english tweets and any that aren't original tweets
def filter(df):
    new_df=df[df.lang == "en"]
    df1 = new_df[new_df['reply'].isnull()]
    df2 = df1[df1['retweet'].isnull()]
    df3 = df2[df2['quote'].isnull()]
    return df3

In [ ]:
#Filter
filtered_politifact_fake_df = filter(df_names)
filtered_politifact_real_df = filter(pr_names)
filtered_gossipcop_fake_df = filter(gf_names)
filtered_gossipcop_real_df = filter(gr_names)

In [ ]:
#Add labels indicating source of ground truth and veracity
filtered_politifact_fake_df['source']='Politifact'
filtered_politifact_fake_df['veracity']='Fake'
filtered_politifact_real_df['source']='Politifact'
filtered_politifact_real_df['veracity']='Real'
filtered_gossipcop_fake_df['source']='Gossipcop'
filtered_gossipcop_fake_df['veracity']='Fake'
filtered_gossipcop_real_df['source']='Gossipcop'
filtered_gossipcop_real_df['veracity']='Real'

In [ ]:
#Keep important columns
df1 = filtered_politifact_fake_df[['text', 'num_retweets', 'source', 'veracity', 
                                   'hashtags', 'followers', 'following', 'tweet_count', 
                                   'mentions', 'urls']].copy()
df2 = filtered_politifact_real_df[['text', 'num_retweets', 'source', 'veracity', 
                                   'hashtags', 'followers', 'following', 'tweet_count', 
                                   'mentions', 'urls']].copy()
df3 = filtered_gossipcop_fake_df[['text', 'num_retweets', 'source', 'veracity', 
                                  'hashtags', 'followers', 'following', 'tweet_count', 
                                  'mentions', 'urls']].copy()
df4 = filtered_gossipcop_real_df[['text', 'num_retweets', 'source', 'veracity', 
                                  'hashtags', 'followers', 'following', 'tweet_count', 
                                  'mentions', 'urls']].copy()

In [ ]:
#Combine all dataframes into one
data = [df1, df2, df3, df4]
fakeNewsNet = pd.concat(data)

In [ ]:
#Save complete dataframe to csv
fakeNewsNet.to_csv('fakeNewsNet.csv', index = True, header=True)

## Data Preprocessing
- Check for missingness
- Count and remove hashtags, mentions and URLs
- Remove emojis, repeated letters, numbers and stopwords from text
- Make text lowercase and lemmatize
- Investigate most popular words

In [ ]:
#Check for nulls - hashtags, mentions and urls will be calculated next
fakeNewsNet.isna().sum()

In [ ]:
#Investigate dataframe
fakeNewsNet.head()

In [ ]:
#Create hashtag count column and remove hashtag column
hashtag_count = []
for i in fakeNewsNet['hashtags']:
    if pd.isna(i):
        hashtag_count.append(0)
    else:
        hashtag_count.append(i.count('tag'))

fakeNewsNet['num_hashtags'] = hashtag_count
fakeNewsNet.drop('hashtags', inplace=True, axis=1)

In [ ]:
#Create mentions count column and remove mention column
mentions_count = []
for i in fakeNewsNet['mentions']:
    if pd.isna(i):
        mentions_count.append(0)
    else:
        mentions_count.append(i.count('username'))

fakeNewsNet['num_mentions'] = mentions_count
fakeNewsNet.drop('mentions', inplace=True, axis=1)

In [ ]:
#Create URL count column and remove URL column
urls_count = []
for i in fakeNewsNet['urls']:
    if pd.isna(i):
        urls_count.append(0)
    else:
        urls_count.append(i.count('"url"'))

fakeNewsNet['num_urls'] = urls_count
fakeNewsNet.drop('urls', inplace=True, axis=1)

In [ ]:
#Check for nulls again - none found
fakeNewsNet.isna().sum()

In [ ]:
#Save english stopwords 
stopwords_english = stopwords.words('english')

#Add extra stopwords as desired
extra_stops = ['via', 'news', 'u', 'e', 'trump', 'amp', 'bieber', 'us']
for i in extra_stops:
    stopwords_english.append(i)

#Initiate lemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
#Function that removes usernames, URLS and \n characters and shortens repeated letters
def abbrev_text(text_list):
    abbrev_text = []

    for i in range(len(text_list)):

        #Twitter handles removed to USERNAME 
        text = re.sub(r"@[\S]+", " ", text_list[i])

        #Links changed to URL 
        text = re.sub(r"(http://[\S]+)|(https://[\S]+)|(www\.[\S]+)", " ", text)

        #Repeated letters curtailed to two
        text = re.sub(r'(\w)\1(\1+)', r'\1\1', text)
        
        #Remove \n characters
        final = text.replace('\\n', '')
        
        abbrev_text.append(final)
    
    return abbrev_text

In [ ]:
#Function to remove emojis 
def remove_emojis(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [ ]:
#Function removes punctuation, numbers, capital letters and stopwords and lemmatizes 
def preprocess(text):
    preprocessed_tokens = []
    #Remove punctuation
    stripped_text = text.translate(str.maketrans('', '', string.punctuation))
    for word in word_tokenize(stripped_text):
        #Remove numbers, stopwords and make lowercase
        word = word.lower()
        if word.isalpha() and word not in stopwords_english:
            preprocessed_tokens.append(lemmatizer.lemmatize(word))
    preprocessed_words = " ".join(preprocessed_tokens)
    return preprocessed_words

In [ ]:
#Clean all text data in dataset
abbrev_text_all = abbrev_text(fakeNewsNet['text'])
clean_text = []

for i in tqdm(range(len(abbrev_text_all))):
    emoji_free = remove_emojis(abbrev_text_all[i])
    preprocessed = preprocess(emoji_free)
    if len(preprocessed)>0:
        clean_text.append(preprocessed)
    else: 
        clean_text.append("EMPTY")
        
fakeNewsNet['clean_text'] = clean_text

In [ ]:
#Most popular words in the train data
words = []

#Make a list of all of the words in all of the tweets
for i in fakeNewsNet['clean_text']:
    for word in word_tokenize(i):
        words.append(word)

#Counter instance
c_tweet = Counter(words)

#Find 20 most popular words
most_popular = c_tweet.most_common(20)

In [ ]:
#Plot 20 most popular words
sns.barplot(x=[count for word, count in most_popular], 
            y=[word for word, count in most_popular])
plt.xlabel('Count')
plt.ylabel('Word')
plt.show()

## Feature Extraction
- Import VAD lexicon and build function to get VAD scores (NRC-EIL scores found with emotion-nrc-affect-lex package)
- Filter out tweets with less than 3 hits on VAD lexicon
- Find length of original tweet in words

In [ ]:
#Import lexicon
nrc_vad_df = pd.read_csv('NRC-VAD-Lexicon.txt', 
                         names=["term", "valence", "arousal", "dominance"], sep='\t')

In [ ]:
#Transform dataframe to dictionary for improved search time
nrc_vad_dict = nrc_vad_df.set_index('term').T.to_dict('list')

In [ ]:
#Function to get vad and emotion scores from text
def emotion(tweet):
    token_tweet = word_tokenize(tweet)
    #Initialize VAD to zero
    valence = 0
    arousal = 0
    dominance = 0
    vad_count = 0
    
    #Search for each word in VAD dictionary and sum values 
    for word in token_tweet:
        vad_list = nrc_vad_dict.get(word, [0, 0, 0])
        if vad_list[2] > 0:
            vad_count += 1
            valence += vad_list[0]
            arousal += vad_list[1]
            dominance += vad_list[2]
    
    #Account for zero division error and average VAD scores
    if vad_count > 0: 
        val_tweet = valence/vad_count
        aro_tweet = arousal/vad_count
        dom_tweet = dominance/vad_count
    else:
        val_tweet = 0
        aro_tweet = 0
        dom_tweet = 0
        
    
    #use emo to get emotion intensities then divide by count for tweet average
    emo_object = Emo(word_tokenize(tweet))
    weight = emo_object.weighted_emotion_scores
    count = emo_object.raw_emotion_scores
    try:
        anger_tweet = weight.get("anger", 0)/count.get("anger", 0)
    except ZeroDivisionError:
        anger_tweet = 0
        
    try:
        anticipation_tweet = weight.get("anticipation", 0)/count.get("anticipation", 0)
    except ZeroDivisionError:
        anticipation_tweet = 0
        
    try:
        disgust_tweet = weight.get("disgust", 0)/count.get("disgust", 0)
    except ZeroDivisionError:
        disgust_tweet = 0
        
    try:
        fear_tweet = weight.get("fear", 0)/count.get("fear", 0)
    except ZeroDivisionError:
        fear_tweet = 0
        
    try:
        joy_tweet = weight.get("joy", 0)/count.get("joy", 0)
    except ZeroDivisionError:
        joy_tweet = 0
        
    try:
        sadness_tweet = weight.get("sadness", 0)/count.get("sadness", 0)
    except ZeroDivisionError:
        sadness_tweet = 0
        
    try:
        surprise_tweet = weight.get("surprise", 0)/count.get("surprise", 0)
    except ZeroDivisionError:
        surprise_tweet = 0
        
    try:
        trust_tweet = weight.get("trust", 0)/count.get("trust", 0)
    except ZeroDivisionError:
        trust_tweet = 0
    
    return val_tweet, aro_tweet, dom_tweet, anger_tweet, anticipation_tweet, disgust_tweet, fear_tweet, joy_tweet, sadness_tweet, surprise_tweet, trust_tweet, vad_count

In [ ]:
#Run through each tweet finding VAD and EIL scores
val_list = []
aro_list = []
dom_list = []
anger_list = []
anticipation_list = []
disgust_list = []
fear_list = []
joy_list = []
sadness_list = []
surprise_list = []
trust_list = []
vad_count_list = []

for tweet in tqdm(fakeNewsNet['clean_text']):
    val_list.append(emotion(tweet)[0])
    aro_list.append(emotion(tweet)[1])
    dom_list.append(emotion(tweet)[2])
    anger_list.append(emotion(tweet)[3])
    anticipation_list.append(emotion(tweet)[4])
    disgust_list.append(emotion(tweet)[5])
    fear_list.append(emotion(tweet)[6])
    joy_list.append(emotion(tweet)[7])
    sadness_list.append(emotion(tweet)[8])
    surprise_list.append(emotion(tweet)[9])
    trust_list.append(emotion(tweet)[10])
    vad_count_list.append(emotion(tweet)[11])

In [ ]:
#Add emotion metric scores to dataframe
fakeNewsNet['valence'] = val_list
fakeNewsNet['arousal'] = aro_list
fakeNewsNet['dominance'] = dom_list
fakeNewsNet['anger'] = anger_list
fakeNewsNet['anticipation'] = anticipation_list
fakeNewsNet['disgust'] = disgust_list
fakeNewsNet['fear'] = fear_list
fakeNewsNet['joy'] = joy_list
fakeNewsNet['sadness'] = sadness_list
fakeNewsNet['surprise'] = surprise_list
fakeNewsNet['trust'] = trust_list
fakeNewsNet['vad_count'] = vad_count_list

In [ ]:
#Investigate dataframe
fakeNewsNet.head()

In [ ]:
#Save to csv
fakeNewsNet.to_csv('fakeNewsNet_emotionscores.csv', index = False, header=True)

In [ ]:
#Count totals
df_count = fakeNewsNet.groupby(['veracity']).count()
df_count[df_count.columns[0]]

In [ ]:
#Filter to only those instances with 3 or more VAD scores
fNN_compact = fakeNewsNet[fakeNewsNet.vad_count > 2].copy()

In [ ]:
#Add length of original text as feature
fNN_compact['tweet_length'] = fNN_compact['text'].apply(lambda x: len(x.split()))

In [ ]:
#Save new smaller dataset
fNN_compact.to_csv('fNN_compact.csv', index = False, header=True)

## Train/Validation/Test Splits

In [ ]:
feature_cols = ['followers', 'following', 'tweet_count', 'num_hashtags', 
                'num_mentions', 'num_urls', 'tweet_length', 'valence', 
                'arousal', 'dominance', 'anger', 'anticipation', 'disgust', 
                'fear', 'joy', 'sadness', 'surprise', 'trust']

#split dataset in features and target variable
X = fNN_compact[feature_cols]
y = fNN_compact['veracity']

#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    stratify=y, random_state=42)

#Split test into test and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, 
                                                    stratify=y_train, random_state=42)

In [ ]:
#Save separate datasets
X_train.to_csv('X_train.csv', index = False, header=True)
X_val.to_csv('X_val.csv', index = False, header=True)
X_test.to_csv('X_test.csv', index = False, header=True)
y_train.to_csv('y_train.csv', index = False, header=True)
y_val.to_csv('y_val.csv', index = False, header=True)
y_test.to_csv('y_test.csv', index = False, header=True)

In [ ]:
#Count totals - X_train
df_train = X_train.join(y_train)
train_count = df_train.groupby(['veracity']).count()
train_count[train_count.columns[0]]

In [ ]:
#Count totals - X_val
df_val = X_val.join(y_val)
val_count = df_val.groupby(['veracity']).count()
val_count[val_count.columns[0]]

In [ ]:
#Count totals - X_test
df_test = X_test.join(y_test)
test_count = df_test.groupby(['veracity']).count()
df_test[test_count.columns[0]]

## Exploratory Data Analysis
- Check correlations
- Univariate analysis visualisations for non-emotion metrics
- Welch's t-tests for emotion metrics
- Effect size for emotion metrics
- Visualisations for emotion metrics

In [ ]:
#Join train X and y into one train dataframe
fNN_train = X_train.join(y_train)

In [ ]:
#Correlations
fNN_corr = fNN_train.drop(['veracity'], axis=1)

sns.heatmap(fNN_corr.corr(), cmap="vlag")
plt.show
fNN_corr.corr()

In [ ]:
#Descriptive statistics
pd.set_option('display.float_format', lambda x: '%.2f' % x)
print(fNN_train.describe())

In [ ]:
#Visualise lengths
sns.set_theme(style="whitegrid")
plt.figure(figsize = (15,8))
sns.countplot(x = fNN_train['tweet_length'], palette="Blues_d")
plt.title('Tweet Length Distribution', fontsize = 12)
plt.xlabel('Words per Tweet', fontsize = 12)
plt.ylabel('Number of Tweets', fontsize = 12)
plt.show()

In [ ]:
#Compare tweet lengths 
sns.violinplot(y=fNN_train["tweet_length"], x=fNN_train["veracity"])
plt.title('Tweet Length by Veracity', fontsize = 12)
plt.ylabel('Words per Tweet', fontsize = 12)
plt.xlabel('Veracity', fontsize = 12)
plt.show()

In [ ]:
#Violin plots for non-emotion metrics - log(n+1) transformed for clearer visualisation
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(12, 12))

#Compare followers
sns.violinplot(ax=ax1, y=np.log10(fNN_train["followers"]+1), x=fNN_train["veracity"])
ax1.set_title('Number of followers by Veracity')
ax1.set_ylabel('log(followers+1)')

#Compare following
sns.violinplot(ax=ax2, y=np.log10(fNN_train["following"]+1), x=fNN_train["veracity"])
ax2.set_title('Number following by Veracity')
ax2.set_ylabel('log(following+1)')

#Compare tweet counts
sns.violinplot(ax= ax3, y=np.log10(fNN_train["tweet_count"]+1), x=fNN_train["veracity"])
ax3.set_title('Tweet count')
ax3.set_ylabel('log(tweet_count+1)')

#Compare number hashtags
sns.violinplot(ax=ax4, y=np.log10(fNN_train["num_hashtags"]+1), x=fNN_train["veracity"])
ax4.set_title('Number of hashtags by veracity')
ax4.set_ylabel('log(num_hashtags+1)')

#Compare number mentions
sns.violinplot(ax=ax5, y=np.log10(fNN_train["num_mentions"]+1), x=fNN_train["veracity"])
ax5.set_title('Number of mentions by veracity')
ax5.set_ylabel('log(num_mentions+1)')

#Compare number urls
sns.violinplot(ax=ax6, y=np.log10(fNN_train["num_urls"]+1), x=fNN_train["veracity"])
ax6.set_title('Number of URLs by veracity')
ax6.set_ylabel('log(num_urls+1)')

fig.tight_layout()
plt.show()

In [ ]:
#Save vad values as series
valence = fNN_train['valence']
arousal = fNN_train['arousal']
dominance = fNN_train['dominance']
fake = fNN_train['veracity'] == 'Fake'

fake_val = valence[fake]
real_val = valence[~fake]
fake_aro = arousal[fake]
real_aro = arousal[~fake]
fake_dom = dominance[fake]
real_dom = dominance[~fake]

In [ ]:
#Welch’s t-tests VAD 
print("Valence:", ttest_ind(fake_val, real_val, equal_var=False))
print("Arousal:", ttest_ind(fake_aro, real_aro, equal_var=False))
print("Dominance:", ttest_ind(fake_dom, real_dom, equal_var=False))

In [ ]:
# Function to calculate Cohen's d for independent samples
def cohend(d1, d2):
    # Calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # Calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    # Calculate the pooled standard deviation
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # Calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)
    # Calculate effect size
    return (u1 - u2) / s

In [ ]:
#Cohen's d VAD 
print("Cohen's d for valence: " + str(cohend(fake_val, real_val)))
print("Cohen's d for arousal: " + str(cohend(fake_aro, real_aro)))
print("Cohen's d for dominance: " + str(cohend(fake_dom, real_dom)))

In [ ]:
#Violin plots for VAD - log(n+1) transformed for clearer visualisation
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 12))

#Compare valence
sns.violinplot(ax=ax1, y=fNN_train["valence"], x=fNN_train["veracity"])
ax1.set_title('Valence')

#Compare arousal
sns.violinplot(ax=ax2, y=fNN_train["arousal"], x=fNN_train["veracity"])
ax2.set_title('Arousal')

#Compare dominance
sns.violinplot(ax=ax3, y=fNN_train["dominance"], x=fNN_train["veracity"])
ax3.set_title('Dominance')

for ax in fig.get_axes():
    ax.set(xlabel='Veracity', ylabel='Score')
    
fig.tight_layout()
plt.show()

In [ ]:
#Save emotion values as series
anger = fNN_train['anger']
anticipation = fNN_train['anticipation']
disgust = fNN_train['disgust']
fear = fNN_train['fear']
joy = fNN_train['joy']
sadness = fNN_train['sadness']
surprise = fNN_train['surprise']
trust = fNN_train['trust']

fake_anger = anger[fake]
real_anger = anger[~fake]
fake_antic = anticipation[fake]
real_antic = anticipation[~fake]
fake_disgust = disgust[fake]
real_disgust = disgust[~fake]
fake_fear = fear[fake]
real_fear = fear[~fake]
fake_joy = joy[fake]
real_joy = joy[~fake]
fake_sadness = sadness[fake]
real_sadness = sadness[~fake]
fake_surprise = surprise[fake]
real_surprise = surprise[~fake]
fake_trust = trust[fake]
real_trust = trust[~fake]

In [ ]:
#Welch’s t-tests for discrete emotion metrics
print("Anger:", ttest_ind(fake_anger, real_anger, equal_var=False))
print("Anticipation:", ttest_ind(fake_antic, real_antic, equal_var=False))
print("Disgust:", ttest_ind(fake_disgust, real_disgust, equal_var=False))
print("Fear:", ttest_ind(fake_fear, real_fear, equal_var=False))
print("Joy:", ttest_ind(fake_joy, real_joy, equal_var=False))
print("Sadness:", ttest_ind(fake_sadness, real_sadness, equal_var=False))
print("Suprise:", ttest_ind(fake_surprise, real_surprise, equal_var=False))
print("Trust:", ttest_ind(fake_trust, real_trust, equal_var=False))

In [ ]:
#Cohen's d for discrete emotion metrics
print("Cohen's d for anger: " + str(cohend(fake_anger, real_anger)))
print("Cohen's d for anticipation: " + str(cohend(fake_antic, real_antic)))
print("Cohen's d for disgust: " + str(cohend(fake_disgust, real_disgust)))
print("Cohen's d for fear: " + str(cohend(fake_fear, real_fear)))
print("Cohen's d for joy: " + str(cohend(fake_joy, real_joy)))
print("Cohen's d for sadness: " + str(cohend(fake_sadness, real_sadness)))
print("Cohen's d for surprise: " + str(cohend(fake_surprise, real_surprise)))
print("Cohen's d for trust: " + str(cohend(fake_trust, real_trust)))

In [ ]:
#Get binary present/not present for discrete emotion metrics
fNN_train['anger_zero'] = np.where(fNN_train['anger']== 0, True, False)
fNN_train['antic_zero'] = np.where(fNN_train['anticipation']== 0, True, False)
fNN_train['disgust_zero'] = np.where(fNN_train['disgust']== 0, True, False)
fNN_train['fear_zero'] = np.where(fNN_train['fear']== 0, True, False)
fNN_train['joy_zero'] = np.where(fNN_train['joy']== 0, True, False)
fNN_train['sadness_zero'] = np.where(fNN_train['sadness']== 0, True, False)
fNN_train['surprise_zero'] = np.where(fNN_train['surprise']== 0, True, False)
fNN_train['trust_zero'] = np.where(fNN_train['trust']== 0, True, False)

old_cols = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 
            'trust']
new_cols = ['anger_positive', 'antic_positive', 'disgust_positive', 'fear_positive', 
            'joy_positive', 'sadness_positive', 'surprise_positive', 'trust_positive']

fNN_train[new_cols] = fNN_train[old_cols]
fNN_train[new_cols] = fNN_train[new_cols].replace(0, np.nan)

In [ ]:
# Violin plots for discrete emotions - log(n+1) transformed for clearer visualisation
#Fig setup
fig,((ax1, ax2),(ax3, ax4),(ax5, ax6),(ax7, ax8)) = plt.subplots(4, 2, figsize=(12, 12))

#Compare anger
sns.violinplot(ax=ax1, y=fNN_train["anger_positive"], x=fNN_train["veracity"])
ax1.set_title('Anger')

#Compare anticipation
sns.violinplot(ax=ax2, y=fNN_train["antic_positive"], x=fNN_train["veracity"])
ax2.set_title('Anticipation')

#Compare disgust
sns.violinplot(ax=ax3, y=fNN_train["disgust_positive"], x=fNN_train["veracity"])
ax3.set_title('Disgust')

#Compare fear
sns.violinplot(ax=ax4, y=fNN_train["fear_positive"], x=fNN_train["veracity"])
ax4.set_title('Fear')

#Compare joy
sns.violinplot(ax=ax5, y=fNN_train["joy_positive"], x=fNN_train["veracity"])
ax5.set_title('Joy')

#Compare disgust
sns.violinplot(ax=ax6, y=fNN_train["sadness_positive"], x=fNN_train["veracity"])
ax6.set_title('Sadness')

#Compare disgust
sns.violinplot(ax=ax7, y=fNN_train["surprise_positive"], x=fNN_train["veracity"])
ax7.set_title('Surprise')

#Compare disgust
sns.violinplot(ax=ax8, y=fNN_train["trust_positive"], x=fNN_train["veracity"])
ax8.set_title('Trust')

for ax in fig.get_axes():
    ax.set(xlabel='Veracity', ylabel='Emotion Score')
    
fig.tight_layout()
plt.show()

In [ ]:
#Compare presence of any level of emotion (binary present or not)
len_fake = (fNN_train['veracity']=='Fake').sum()
len_real = (fNN_train['veracity']=='Real').sum()

emotions = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 
            'trust']
zeroes = ['anger_zero', 'antic_zero', 'disgust_zero', 'fear_zero', 'joy_zero', 
          'sadness_zero', 'surprise_zero', 'trust_zero']
percentage_fake = []
percentage_real = []

for i in zeroes:
    count = fNN_train.groupby([i,'veracity']).count()
    count[count.columns[0]]
    percent_fake = count[count.columns[0]][0][0]/len_fake*100
    percentage_fake.append(percent_fake)
    percent_real = count[count.columns[0]][0][1]/len_real*100
    percentage_real.append(percent_real)

#Percentage of tweets containing each discrete emotion
percent_df = pd.DataFrame(list(zip(emotions, percentage_fake, percentage_real)), 
                          columns =['emotion', 'fake', 'real'])

In [ ]:
#Plot discrete emotion tweet percentages
percent_df.plot(x="emotion", y=["fake", "real"], kind="bar",figsize=(9,8))
plt.ylabel('Percentage')
plt.xlabel('Emotion')
plt.legend(title="Veracity", fontsize='small', fancybox=True)
plt.title('Percentage of tweets that contain each emotion')
plt.show()

## Hypothesis Testing
- Check correlations
- Develop logistic regression models with and without emotion metric features
- Likelihood-ratio test

In [ ]:
#Change veracity labels to binary 0/1
y_train = y_train['veracity'].map({'Real': 0, 'Fake': 1})
y_val = y_val['veracity'].map({'Real': 0, 'Fake': 1})
y_test = y_test['veracity'].map({'Real': 0, 'Fake': 1})

In [ ]:
#Add constant to provide intercept
X_train = sm.add_constant(X_train)
X_val = sm.add_constant(X_val)

In [ ]:
#Logistic regression with all variables
logit = sm.Logit(endog = y_train, exog = X_train)
full_model = logit.fit()
full_ll = full_model.llf
print(full_model.summary())

In [ ]:
#Partial logistic regression model without emotions
partial_cols = ['followers', 'following', 'tweet_count', 'num_hashtags', 'num_mentions', 
                'num_urls', 'tweet_length']

X_train_partial = X_train[partial_cols]

logit_p = sm.Logit(endog = y_train, exog = X_train_partial)
partial_model = logit_p.fit()
partial_ll = partial_model.llf
print(partial_model.summary())

In [ ]:
#Log-likelihood test
print("Partial LL:", partial_ll)
print("Full LL:", full_ll)
#calculate likelihood ratio test statistic
LR_statistic = -2*(partial_ll - full_ll)

#Calculate p-value of test statistic using 11 degrees of freedom 
# 11 is difference between models' degrees of freedom
p_val = scipy.stats.chi2.sf(LR_statistic, 11)

print('LR test, p value: {:.2f}, {:.4f}'.format(LR_statistic, p_val))

##### Statistically significant (p<0.0001) so reject null - model with emotion metric features fits data better than model without 

## Models
- Standardize feature values
- For each of logistic regression with stochastic gradient descent (SGD), decision tree, random forest and XGBoost:
    - Grid search optimisation
    - Train model with optimal hyperparameters
    - Test on testing dataset to get confusion matrix
    - Bootstrap testing dataset to get confidence intervals for evaluation metrics
- ROC and precision-recall curves

In [ ]:
#Standardize
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_val_std = scaler.transform(X_val)
X_test_std = scaler.transform(X_test)

## Logistic Regression with SGD

In [ ]:
#Grid search for optimal parameters 
params = {
    "alpha" : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
    "penalty" : ["l2", "l1", "elasticnet"],
}

sgd = SGDClassifier(class_weight="balanced", loss="log")
GS_sgd = GridSearchCV(sgd, param_grid=params, cv=3, scoring='average_precision')

GS_sgd.fit(X_train_std, y_train)

In [ ]:
print(GS_sgd.best_params_) 

##### Optimal parameters: {'alpha': 0.01, 'penalty': 'l2'}**

In [ ]:
logreg_model = SGDClassifier(class_weight="balanced", 
                             loss="log", 
                             alpha=0.01, 
                             penalty="l2")

logreg_model.fit(X_train_std, y_train)

In [ ]:
#Predict with optimal hyperparameters
logreg_y_pred = logreg_model.predict(X_test_std) 

In [ ]:
#Confusion Matrix
logreg_cnf_matrix = confusion_matrix(y_test, logreg_y_pred)
#heatmap
ax1 = sns.heatmap(pd.DataFrame(logreg_cnf_matrix), annot=True, cmap="YlGnBu", fmt='g')
plt.ylabel('Actual')
plt.xlabel('Predicted')
ax1.set_yticklabels(["Real", "Fake"])
ax1.set_xticklabels(["Real", "Fake"])
ax1.set_title("LogReg Confusion Matrix")
plt.show()

In [ ]:
# Classification Report
print(classification_report(y_test, logreg_y_pred))

In [ ]:
#Function to print mean and 95% confidence intervals for bootstrapped test data
def summary_stats(accuracy, precision, recall, f1, pr_auc):
    
    #Accuracy
    mean_acc = np.mean(accuracy)
    lower_acc = np.percentile(accuracy, 2.5)
    upper_acc = np.percentile(accuracy, 97.5)

    #Precision
    mean_prec = np.mean(precision)
    lower_prec = np.percentile(precision, 2.5)
    upper_prec = np.percentile(precision, 97.5)

    #Recall
    mean_rec = np.mean(recall)
    lower_rec = np.percentile(recall, 2.5)
    upper_rec = np.percentile(recall, 97.5)

    #F1
    mean_f1 = np.mean(f1)
    lower_f1= np.percentile(f1, 2.5)
    upper_f1 = np.percentile(f1, 97.5)

    #PR-AUC
    mean_auc = np.mean(pr_auc)
    lower_auc = np.percentile(pr_auc, 2.5)
    upper_auc = np.percentile(pr_auc, 97.5)
     

    print("Mean Accuracy: %.3f" % (mean_acc), "-- CI: %.3f" % (lower_acc), 
          "- %.3f" % (upper_acc))
    print("Mean Precision: %.3f" % (mean_prec), "-- CI: %.3f" % (lower_prec), 
          "- %.3f" % (upper_prec))
    print("Mean Recall: %.3f" % (mean_rec), "-- CI: %.3f" % (lower_rec), 
          "- %.3f" % (upper_rec))
    print("Mean F1: %.3f" % (mean_f1), "-- CI: %.3f" % (lower_f1), 
          "- %.3f" % (upper_f1))
    print("Mean PR-AUC: %.3f" % (mean_auc), "-- CI: %.3f" % (lower_auc), 
          "- %.3f" % (upper_auc))

In [ ]:
#Bootstrap predictions to get confidence intervals
rng = np.random.RandomState(seed=42)
index = np.arange(y_test.shape[0])
logreg_boot_accs = []
logreg_boot_precs = []
logreg_boot_recs = []
logreg_boot_f1s = []
logreg_boot_pr_aucs = []

for i in range(200):
    #Bootstrap sample
    pred_index = rng.choice(index, size=index.shape[0], replace=True)
    #Accuracy
    logreg_boot_acc = metrics.accuracy_score(y_test[pred_index], 
                                             logreg_y_pred[pred_index])
    logreg_boot_accs.append(logreg_boot_acc)
    #Precision
    logreg_boot_prec = metrics.precision_score(y_test[pred_index], 
                                               logreg_y_pred[pred_index])
    logreg_boot_precs.append(logreg_boot_prec)
    #Recall
    logreg_boot_rec = metrics.recall_score(y_test[pred_index], 
                                           logreg_y_pred[pred_index])
    logreg_boot_recs.append(logreg_boot_rec)
    #F1
    logreg_boot_f1 = metrics.f1_score(y_test[pred_index], 
                                      logreg_y_pred[pred_index])
    logreg_boot_f1s.append(logreg_boot_f1)    
    #AUC-PR
    logreg_precision, logreg_recall, _ = precision_recall_curve(y_test[pred_index], 
                                                                logreg_prob[pred_index])
    logreg_pr_auc = auc(logreg_recall, logreg_precision)
    logreg_boot_pr_aucs.append(logreg_pr_auc)
    
summary_stats(logreg_boot_accs, logreg_boot_precs, logreg_boot_recs, 
              logreg_boot_f1s, logreg_boot_pr_aucs)

## Decision Tree

In [ ]:
#Grid search for optimal parameters 
params = {
    "criterion" : ["gini", "entropy" ],
    "max_depth" : [18, 19, 20, 21, 22],
    "min_samples_split": [8, 9, 10, 11, 12, 13],
    "min_samples_leaf": [4, 5, 6, 7, 8]
}

dt = DecisionTreeClassifier()
GS_dt = GridSearchCV(dt, param_grid=params, scoring='average_precision', cv=3)

GS_dt.fit(X_train_std, y_train)

In [ ]:
print(GS_dt.best_params_) 

##### Optimal parameters: {'criterion': 'gini', 'max_depth': 22, 'min_samples_leaf': 8, 'min_samples_split': 9}

In [ ]:
dt_model = DecisionTreeClassifier(criterion='gini',
                                  max_depth=22, 
                                  min_samples_leaf=8,
                                  min_samples_split=9)

dt_model.fit(X_train_std, y_train)

In [ ]:
#Predict
dt_y_pred = dt_model.predict(X_test_std)

In [ ]:
#Confusion Matrix
dt_cnf_matrix = confusion_matrix(y_test, dt_y_pred)
#heatmap
ax1 = sns.heatmap(pd.DataFrame(dt_cnf_matrix), annot=True, cmap="YlGnBu", fmt='g')
plt.ylabel('Actual')
plt.xlabel('Predicted')
ax1.set_yticklabels(["Real", "Fake"])
ax1.set_xticklabels(["Real", "Fake"])
ax1.set_title("Decision Tree Confusion Matrix")
plt.show()

In [ ]:
# Classification Report
print(classification_report(y_test, dt_y_pred))

In [ ]:
#Bootstrap predictions to get confidence intervals
rng = np.random.RandomState(seed=42)
index = np.arange(y_test.shape[0])
dt_boot_accs = []
dt_boot_precs = []
dt_boot_recs = []
dt_boot_f1s = []
dt_boot_pr_aucs = []

for i in range(200):
    #Bootstrap sample
    pred_index = rng.choice(index, size=index.shape[0], replace=True)
    #Accuracy
    dt_boot_acc = metrics.accuracy_score(y_test[pred_index], dt_y_pred[pred_index])
    dt_boot_accs.append(dt_boot_acc)
    #Precision
    dt_boot_prec = metrics.precision_score(y_test[pred_index], dt_y_pred[pred_index])
    dt_boot_precs.append(dt_boot_prec)
    #Recall
    dt_boot_rec = metrics.recall_score(y_test[pred_index], dt_y_pred[pred_index])
    dt_boot_recs.append(dt_boot_rec)
    #F1
    dt_boot_f1 = metrics.f1_score(y_test[pred_index], dt_y_pred[pred_index])
    dt_boot_f1s.append(dt_boot_f1)    
    #AUC-PR
    dt_precision, dt_recall, _ = precision_recall_curve(y_test[pred_index], 
                                                        dt_prob[pred_index])
    dt_pr_auc = auc(dt_recall, dt_precision)
    dt_boot_pr_aucs.append(dt_pr_auc)
    
summary_stats(dt_boot_accs, dt_boot_precs, dt_boot_recs, dt_boot_f1s, dt_boot_pr_aucs)

## Random Forest

In [ ]:
#Grid search for optimal parameters 
params = { 
    'n_estimators': [100, 200, 500],
    'max_features': ['log2', 'sqrt'],
    'max_depth' : [3, 5, 7, None],
    'criterion' :['gini', 'entropy']
}


rf = RandomForestClassifier()
GS_rf = GridSearchCV(rf, param_grid=params, scoring='average_precision', cv=3)

GS_rf.fit(X_train_std, y_train)

In [ ]:
print(GS_rf.best_params_) 

##### Optimal parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 500}

In [ ]:
rf_model = RandomForestClassifier(criterion='entropy',
                            max_depth=None,
                            max_features='sqrt',
                            n_estimators=500)

rf_model.fit(X_train_std, y_train())

In [ ]:
#Predict
rf_y_pred = rf_model.predict(X_test_std)

In [ ]:
#Confusion Matrix
rf_cnf_matrix = confusion_matrix(y_test, rf_y_pred)
#heatmap
ax1 = sns.heatmap(pd.DataFrame(rf_cnf_matrix), annot=True, cmap="YlGnBu", fmt='g')
plt.ylabel('Actual')
plt.xlabel('Predicted')
ax1.set_yticklabels(["Real", "Fake"])
ax1.set_xticklabels(["Real", "Fake"])
ax1.set_title("Random Forest Confusion Matrix")
plt.show()

In [ ]:
# Classification Report
print(classification_report(y_test, rf_y_pred))

In [ ]:
#Bootstrap predictions to get confidence intervals
rng = np.random.RandomState(seed=42)
index = np.arange(y_test.shape[0])
rf_boot_accs = []
rf_boot_precs = []
rf_boot_recs = []
rf_boot_f1s = []
rf_boot_pr_aucs = []

for i in range(200):
    
    #Bootstrap sample
    pred_index = rng.choice(index, size=index.shape[0], replace=True)
    #Accuracy
    rf_boot_acc = metrics.accuracy_score(y_test[pred_index], rf_y_pred[pred_index])
    rf_boot_accs.append(rf_boot_acc)
    #Precision
    rf_boot_prec = metrics.precision_score(y_test[pred_index], rf_y_pred[pred_index])
    rf_boot_precs.append(rf_boot_prec)
    #Recall
    rf_boot_rec = metrics.recall_score(y_test[pred_index], rf_y_pred[pred_index])
    rf_boot_recs.append(rf_boot_rec)
    #F1
    rf_boot_f1 = metrics.f1_score(y_test[pred_index], rf_y_pred[pred_index])
    rf_boot_f1s.append(rf_boot_f1)    
    #AUC-PR
    rf_precision, rf_recall, _ = precision_recall_curve(y_test[pred_index], 
                                                        rf_prob[pred_index])
    rf_pr_auc = auc(rf_recall, rf_precision)
    rf_boot_pr_aucs.append(rf_pr_auc)
    
summary_stats(rf_boot_accs, rf_boot_precs, rf_boot_recs, rf_boot_f1s, rf_boot_pr_aucs)

## XGBoost

In [ ]:
params = {
    'max_depth': [15, 16, 17, 18, 19],
    'learning_rate': [0.1, 0.01, 0.05],
    'gamma': [0, 0.25, 1.0],
    'reg_lambda': [0, 1.0, 10.0],
    'scale_pos_weight': [1, 3, 5] 
}

GS_xgb = GridSearchCV(estimator=xgb.XGBClassifier(objective='binary:logistic', 
                                                  use_label_encoder=False),
                      param_grid=params,
                      scoring='average_precision', 
                      n_jobs = -1,
                      cv = 3)

GS_xgb.fit(X_train_std, y_train, 
           early_stopping_rounds=10,                
           eval_metric='aucpr',
           eval_set=[(X_val_std, y_val)])

In [ ]:
print(GS_xgb.best_params_)

##### Optimal parameters: {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 17, 'reg_lambda': 0, 'scale_pos_weight': 1}

In [ ]:
xgb_model = xgb.XGBClassifier(objective='binary:logistic', 
                              use_label_encoder=False, 
                              gamma=0, 
                              learning_rate=0.1, 
                              max_depth=17, 
                              reg_lambda=0, 
                              scale_pos_weight=1)

xgb_model.fit(X_train_std, y_train, 
              early_stopping_rounds=10,                
              eval_metric='aucpr',
              eval_set=[(X_val_std, y_val)])

In [ ]:
#Predict
xgb_y_pred = xgb_model.predict(X_test_std)

In [ ]:
#Confusion Matrix
xgb_cnf_matrix = confusion_matrix(y_test, xgb_y_pred)
#heatmap
ax1 = sns.heatmap(pd.DataFrame(xgb_cnf_matrix), annot=True, cmap="YlGnBu", fmt='g')
plt.ylabel('Actual')
plt.xlabel('Predicted')
ax1.set_yticklabels(["Real", "Fake"])
ax1.set_xticklabels(["Real", "Fake"])
ax1.set_title("XGBoost Confusion Matrix")
plt.show()

In [ ]:
# Classification Report
print(classification_report(y_test, xgb_y_pred))

In [ ]:
#Bootstrap predictions to get confidence intervals
rng = np.random.RandomState(seed=42)
index = np.arange(y_test.shape[0])
xgb_boot_accs = []
xgb_boot_precs = []
xgb_boot_recs = []
xgb_boot_f1s = []
xgb_boot_pr_aucs = []

for i in range(200):
    #Bootstrap sample
    pred_index = rng.choice(index, size=index.shape[0], replace=True)
    #Accuracy
    xgb_boot_acc = metrics.accuracy_score(y_test[pred_index], xgb_y_pred[pred_index])
    xgb_boot_accs.append(xgb_boot_acc)
    #Precision
    xgb_boot_prec = metrics.precision_score(y_test[pred_index], xgb_y_pred[pred_index])
    xgb_boot_precs.append(xgb_boot_prec)
    #Recall
    xgb_boot_rec = metrics.recall_score(y_test[pred_index], xgb_y_pred[pred_index])
    xgb_boot_recs.append(xgb_boot_rec)
    #F1
    xgb_boot_f1 = metrics.f1_score(y_test[pred_index], xgb_y_pred[pred_index])
    xgb_boot_f1s.append(xgb_boot_f1)    
    #AUC-PR
    xgb_precision, xgb_recall, _ = precision_recall_curve(y_test[pred_index], 
                                                          xgb_prob[pred_index])
    xgb_pr_auc = auc(xgb_recall, xgb_precision)
    xgb_boot_pr_aucs.append(xgb_pr_auc)
    
summary_stats(xgb_boot_accs, xgb_boot_precs, xgb_boot_recs, 
              xgb_boot_f1s, xgb_boot_pr_aucs)

In [ ]:
#ROC Curves
#Predicting only majority class
no_model_pred = [0 for i in range(len(y_test))]
#ROC curve for no model
no_model_fpr, no_model_tpr, _ = roc_curve(y_test, no_model_pred)
#AUC for no model
no_model_auc = roc_auc_score(y_test, no_model_pred)

#Calculate ROC curves for models
#Predict probabilities
sgd_prob = sgd_model.predict_proba(X_test_std)[:, 1]
dt_prob = dt_model.predict_proba(X_test_std)[:, 1]
rf_prob = rf_model.predict_proba(X_test_std)[:, 1]
xgb_prob = xgb_model.predict_proba(X_test_std)[:, 1]

#Calculate ROC AUC
sgd_auc = roc_auc_score(y_test, sgd_prob)
dt_auc = roc_auc_score(y_test, dt_prob)
rf_auc = roc_auc_score(y_test, rf_prob)
xgb_auc = roc_auc_score(y_test, xgb_prob)

#Calculate true positive and false positive rate
sgd_fpr, sgd_tpr, _ = roc_curve(y_test, sgd_prob)
dt_fpr, dt_tpr, _ = roc_curve(y_test, dt_prob)
rf_fpr, rf_tpr, _ = roc_curve(y_test, rf_prob)
xgb_fpr, xgb_tpr, _ = roc_curve(y_test, xgb_prob)

#calculate precision and recall
no_model_precision, no_model_recall, _ = precision_recall_curve(y_test, no_model_pred)
sgd_precision, sgd_recall, _ = precision_recall_curve(y_test, sgd_prob)
dt_precision, dt_recall, _ = precision_recall_curve(y_test, dt_prob)
rf_precision, rf_recall, _ = precision_recall_curve(y_test, rf_prob)
xgb_precision, xgb_recall, _ = precision_recall_curve(y_test, xgb_prob)

#Calculate AUC-PR
no_model_pr_auc = auc(no_model_recall, no_model_precision)
sgd_pr_auc = auc(sgd_recall, sgd_precision)
dt_pr_auc = auc(dt_recall, dt_precision)
rf_pr_auc = auc(rf_recall, rf_precision)
xgb_pr_auc = auc(xgb_recall, xgb_precision)

In [ ]:
#ROC curve labels
no_model_label = 'No model: AUC=%.2f' % (no_model_auc)
sgd_label = 'Logistic SGD: AUC=%.2f' % (sgd_auc)
dt_label = 'Decision Tree: AUC=%.2f' % (dt_auc)
rf_label = 'Random Forest: AUC=%.2f' % (rf_auc)
xgb_label = 'XGBoost: AUC=%.2f' % (xgb_auc)

#Plot ROC curves for models
plt.plot(no_model_fpr, no_model_tpr, linestyle='--', label=no_model_label)
plt.plot(sgd_fpr, sgd_tpr, marker='.', label= sgd_label)
plt.plot(dt_fpr, dt_tpr, marker='.', label= dt_label)
plt.plot(rf_fpr, rf_tpr, marker='.', label= rf_label)
plt.plot(xgb_fpr, xgb_tpr, marker='.', label= xgb_label)

#Axes labels and legend
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()

plt.show()

In [ ]:
#PR curve labels
no_model_label = 'No model: AUC-PR=%.2f' % (no_model_pr_auc)
sgd_label = 'Logistic SGD: AUC-PR=%.2f' % (sgd_pr_auc)
dt_label = 'Decision Tree: AUC-PR=%.2f' % (dt_pr_auc)
rf_label = 'Random Forest: AUC-PR=%.2f' % (rf_pr_auc)
xgb_label = 'XGBoost: AUC-PR=%.2f' % (xgb_pr_auc)


#Plot PR curve for models
fig, ax = plt.subplots()
ax.plot(sgd_recall, sgd_precision, color='orange', label=sgd_label)
ax.plot(dt_recall, dt_precision, color='green', label=dt_label)
ax.plot(rf_recall, rf_precision, color='red', label=rf_label)
ax.plot(xgb_recall, xgb_precision, color='purple', label=xgb_label)

no_model = len(y_test[y_test==1]) / len(y_test)
ax.plot([0, 1], [no_model, no_model], linestyle='--', label=no_model_label)

#add axis labels to plot
ax.set_title('Precision-Recall Curve')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')

# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.legend()
#display plot
plt.show()

## Feature importance
- SHAP Values

In [ ]:
X_test_std = pd.DataFrame(X_test_std, columns=X_train.columns)

In [ ]:
explainer = shap.Explainer(xgb_model)
shap_values = explainer(X_test_std)

In [ ]:
#Absolute means
shap.plots.bar(shap_values.abs.mean(0), max_display=19)

In [ ]:
#Calculate standard errors for absolute means
shap_df = pd.DataFrame(shap_values.values, columns=X_test_std.columns)

for column in shap_df.columns:
    print(column, "SHAP SE: %.4f" % (sem(shap_df[column])))

In [ ]:
# summarize the effects of all the features
shap.plots.beeswarm(shap_values, max_display=19, show=False)
plt.gcf().axes[-1].set_aspect('auto')
plt.tight_layout()
# As mentioned, smaller "box_aspect" value to make colorbar thicker
plt.gcf().axes[-1].set_box_aspect(100) 